In [8]:
# import python modules, if it doesn't work, you might need to install them first (e.g. using anaconda)
# %matplotlib inline 
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import os
import matplotlib.gridspec as gridspec
import matplotlib.lines as mlines
import seaborn as sns
import subprocess
import PyCO2SYS as pyco2
import pandas as pd
import copy
import netCDF4 as nc

In [2]:
# dictionaries with info for plotting stuff
lab_dict = {
    'OPSIA_max':'AMOC',
    'ATMT_ALTI':'SAT',
    'OPSIP_min':'IPMOC',
    'OPSIS_min':'SOMOC',
    'OPSI_max':'MOC',
    
    'tinv_TEMP':'T$_{ocean}$',
    'tinv_S':'S',
    'ida':'IDA',
    'tinv_DIC':'DIC',
    'tinv_DIC13':'DIC13',
    'tinv_DIC14':'DIC14',
    'tinv_ALK':'ALK',
    'tinv_PO4':'PO$_4$',
    'tinv_O2':'O$_2$',
    'tinv_SiO':'SiO',
    'tinv_Fe':'Fe',
    'tinv_N2O':'N$_2$O',
    'ICEA':'Sea ice area',
    'd13C_deep_pacific_mean':'Pacific benthic $\delta^{13}$C$_{DIC}$',
    'delta_d13C_P_halfNA':'$\Delta\delta^{13}$C$_{P-NA/2}$',
    'aofluxCO2_SO':'air-sea flux CO$_2$',
    
    'stock_calc':'CaCO$_3$',
    'stock_poc':'POC',
    'stock_opal':'Opal',
    
    'cac13':'$\delta^{13}$C$_{CaCO_3}$',
    'poc13':'$\delta^{13}$C$_{POC}$',
    
    'POM_EXPORT':'POM$_{export}$',
    'CACO3_EXPORT':'CaCO$_{3, export}$',
    'OPAL_EXPORT':'Opal$_{export}$',
    
    'burial_calc':'CaCO$_{3, burial}$',
    'CACO3_MAR':'CaCO$_{3}$ MAR',
    'C_excess':'Excess C burial',
    'OPSIA':'AMOC',
    'So_max_age':'max age in Southern Ocean',
    'So_deep_T':'Southern deep Ocean T',
    'So_deep_S':'Southern deep Ocean S',
    'PO4pre_PO4':'PO$_{4,pre}$/PO$_4$',
    'DIC':'[DIC]',
    'ALK':'[ALK]',
    'CO3':'[CO$_3^{2-}$]',
    
    'atmpco2':'CO$_2$',
    'Datmpco2':'$\Delta$CO$_2$',
    'oceanC':'DIC',
    'd13catm':'$\delta^{13}$C$_{atm}$',
    'd13cdic':'$\delta^{13}$C$_{DIC}$',
    'D14catm':'$\Delta^{14}$C$_{atm}$',
    'D14cdic':'$\Delta^{14}$C$_{DIC}$',
    'D14cdic2':'$\Delta^{14}$C$_{DIC,2km}$',
    'D14cdic3':'$\Delta^{14}$C$_{DIC,3km}$',
    'D14age':'$^{14}$Cage',
    'D14agetest':'$^{14}$Cage',
    'D14age2km':'$^{14}$Cage$_{2km}$',
    'D14age3km':'$^{14}$Cage$_{3km}$',
    'D14agesurf':'$^{14}$Cage$_{100-200m}$',
}
unit_dict = {
    'OPSIA_max':'[Sv]',
    'ATMT_ALTI':'[K]',
    'OPSIP_min':'[-Sv]',
    'OPSIS_min':'[-Sv]',
    'OPSI_max':'[Sv]',
    
    'tinv_TEMP':'[$^\circ$C]',
    'tinv_S':'[PSU]',
    'ida':'(yr)',
    'tinv_DIC':'[mmol m$^{-3}$]',
    'tinv_DIC13':'[mmol m$^{-3}$]',
    'tinv_DIC14':'[mmol m$^{-3}$]',
    'tinv_ALK':'[mmol m$^{-3}$]',
    'tinv_PO4':'[mmol m$^{-3}$]',
    'tinv_O2':'[mmol m$^{-3}$]',
    'tinv_SiO':'[mmol m$^{-3}$]',
    'tinv_Fe':'[$\mu$mol m$^{-3}$]',
    'tinv_N2O':'[$\mu$mol m$^{-3}$]',
    'ICEA':'[km$^{2}$]',
    'd13C_deep_pacific_mean':'[\u2030]',
    'delta_d13C_P_halfNA':'[\u2030]',
    'aofluxCO2_SO':'[mol/(m$^2$s)]',
    
    'stock_calc':'[GtC]',
    'stock_poc':'[GtC]',
    'stock_opal':'[Tmol Si]',
    
    'cac13':'[\u2030]',
    'poc13':'[\u2030]',
    
    'POM_EXPORT':'[GtC yr$^{-1}$]',
    'CACO3_EXPORT':'[GtC yr$^{-1}$]',
    'OPAL_EXPORT':'[Tmol Si yr$^{-1}$]',
    
    'burial_calc':'[GtC yr$^{-1}$]',
    'CACO3_MAR':'(Pg/a)',
    'C_excess':'(GtC yr$^{-1}$)',
    'OPSIA':'(Sv)',
    'So_max_age':'(yr)',
    'So_deep_T':'($^\circ$C)',
    'So_deep_S':'PSU',
    'PO4pre_PO4':'(%)',
    'DIC':'mmol/m$^3$',
    'ALK':'mmol/m$^3$',
    'CO3':'mmol/m$^3$',
    
    'atmpco2':'[ppm]',
    'Datmpco2':'[ppm]',
    'oceanC':'[GtC]',
    'd13catm':'[\u2030]',
    'd13cdic':'[\u2030]',
    'D14catm':'[\u2030]',
    'D14cdic':'[\u2030]',
    'D14cdic2':'[\u2030]',
    'D14cdic3':'[\u2030]',
    'D14age':'[yr]',
    'D14agetest':'[yr]',
    'D14age2km':'[yr]',
    'D14age3km':'[yr]',
    'D14agesurf':'[yr]',
}
fact_dict = {
    'OPSIA_max':1.,
    'ATMT_ALTI':1.,
    'OPSIP_min':-1.,
    'OPSIS_min':-1.,
    'OPSI_max':1.,
    
    'tinv_TEMP':1.,
    'tinv_S':1.,
    'ida':1.,
    'tinv_DIC':1.e3,
    'tinv_DIC13':1.e3,
    'tinv_DIC14':1.e3,
    'tinv_ALK':1.e3,
    'tinv_PO4':1.e3,
    'tinv_O2':1.e3,
    'tinv_SiO':1.e3,
    'tinv_Fe':1.e6,
    'tinv_N2O':1.e6,
    'ICEA':1.,
    'd13C_deep_pacific_mean':1.,
    'delta_d13C_P_halfNA':1.,
    'aofluxCO2_SO':1.,
    
    'stock_calc':1.,
    'stock_poc':1.,
    'stock_opal':1.,
    
    'cac13':1.,
    'poc13':1.,
    
    'POM_EXPORT':1.,
    'CACO3_EXPORT':1.,
    'OPAL_EXPORT':1.,
    
    'burial_calc':1.,
    'burial_poc':1.,
    'CACO3_MAR':1.,
    'C_excess':1.,
    'OPSIA':1.,
    'So_max_age':1.,
    'So_deep_T':1.,
    'So_deep_S':1.,    
    'PO4pre_PO4':100.,
    'O2_LCDW_mean':1.,
    'DIC':1e3,
    'ALK':1e3,
    'CO3':1e3,
    
    'atmpco2':1.,
    'Datmpco2':1.,
    'oceanC':12.011*1.35449639e+18*1.e-15,
    'd13catm':1.,
    'd13cdic':1.,
    'D14catm':1.,
    'D14cdic':1.,
    'D14cdic2':1.,
    'D14cdic3':1.,
    'D14age':1.,
    'D14agetest':1.,
    'D14age2km':1.,
    'D14age3km':1.,
    'D14agesurf':1.,
}

In [3]:
# path to folder
path = './'
# step_changes
NA_full = xr.open_dataset(path+'NA_3d.nc', decode_times=False)
NP_full = xr.open_dataset(path+'NP_3d.nc', decode_times=False)
EP_full = xr.open_dataset(path+'EP_3d.nc', decode_times=False)
SO_full = xr.open_dataset(path+'SO_3d.nc', decode_times=False)
GL_full = xr.open_dataset(path+'GL_3d.nc', decode_times=False)

NA_C_full = xr.open_dataset(path+'NA_C_3d.nc', decode_times=False)
NP_C_full = xr.open_dataset(path+'NP_C_3d.nc', decode_times=False)
EP_C_full = xr.open_dataset(path+'EP_C_3d.nc', decode_times=False)
SO_C_full = xr.open_dataset(path+'SO_C_3d.nc', decode_times=False)
GL_C_full = xr.open_dataset(path+'GL_C_3d.nc', decode_times=False)

# glacial_cycles
NA_longruns_full = xr.open_dataset(path+'NA_longruns_3d.nc', decode_times=False)
NP_longruns_full = xr.open_dataset(path+'NP_longruns_3d.nc', decode_times=False)
EP_longruns_full = xr.open_dataset(path+'EP_longruns_3d.nc', decode_times=False)
SO_longruns_full = xr.open_dataset(path+'SO_longruns_3d.nc', decode_times=False)
GL_longruns_full = xr.open_dataset(path+'GL_longruns_3d.nc', decode_times=False)

NA_longruns_C_full = xr.open_dataset(path+'NA_longruns_C_3d.nc', decode_times=False)
NP_longruns_C_full = xr.open_dataset(path+'NP_longruns_C_3d.nc', decode_times=False)
EP_longruns_C_full = xr.open_dataset(path+'EP_longruns_C_3d.nc', decode_times=False)
SO_longruns_C_full = xr.open_dataset(path+'SO_longruns_C_3d.nc', decode_times=False)
GL_longruns_C_full = xr.open_dataset(path+'GL_longruns_C_3d.nc', decode_times=False)

/storage/mirrored/homes/adloff/miniconda3/envs/jupyter/lib/python3.9/site-packages/xarray/backends/plugins.py:71: RuntimeWarning: Engine 'gini' loading failed:
cannot import name 'Self' from 'typing_extensions' (/storage/mirrored/homes/adloff/miniconda3/envs/jupyter/lib/python3.9/site-packages/typing_extensions.py)
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


In [19]:
def timeseries_short_alkdicerrors_stack_sc(times,alk,dic,co3,ph,s,t,pr,po4,sio,po4reg,aou,tou,o2,d13c,region,labeli,opt):
    colors = {'base':'olive','kgas':'blue','sowi':'lightblue','aerosol':'gray','po4':'xkcd:burnt yellow','pipo':'xkcd:earth','landc':'xkcd:tea','remi':'xkcd:greenish beige','co2t':'cyan','bgc':'xkcd:iris','phys':'xkcd:carmine','allO':'xkcd:gray pink','all':'xkcd:gray pink','fw':'lightblue'}
    labels = {'ctrl':'base','kgas':'kgas','sowi':'sowi','dust':'aerosol','remin':'remi','phos':'po4','caco':'pipo','land':'landc','co2t':'co2t','phys':'phys','bgc':'bgc','allC':'all','fw':'fw','rad':'rad'}
    alphas = [0.7,0.4,1]
    fig = plt.subplots(figsize=(15,5))
    gs = mpl.gridspec.GridSpec(1,3)
    ax = plt.subplot(gs[0,0],xmargin=.5)
    ax2 = plt.subplot(gs[0,1],sharey=ax)
    ax3 = plt.subplot(gs[0,2],sharey=ax)
    axes = [ax,ax2,ax3]
    for i in range(len(alk)):
        if labeli[i][:4] == 'dust':
            for j in range(len(alk[i])):
                dic_change = dic[i][j]-dic[i][j][0]
                alk_change = alk[i][j]-alk[i][j][0]
                co3_change = co3[i][j]-co3[i][j][0]
                dic_sc = []
                pCO2_sc = []
                hco3_sc = []
                alk_sc = []
                OmegaA_sc = []
                OmegaC_sc = []
                for k in range(len(t[i][j])):
                    output = pyco2.sys(par1=ph[i][j][k], par1_type=3, opt_pH_scale=1, par2=co3[i][j][k]*1e3, par2_type=6,temperature=t[i][j][k],salinity=s[i][j][k],pressure=pr[i][j][k]/10000.,total_phosphate=po4[i][j][k]*1e3,total_silicate=sio[i][j][k]*1e3,opt_k_carbonic=14,opt_total_borate=2)
                    dic_sc.append(output['dic']*1e-3)
                    pCO2_sc.append(output['pCO2'])
                    CO2_sc.append(output['aqueous_CO2']*1e-3)
                    hco3_sc.append(output['HCO3']*1e-3)
                    alk_sc.append(output['alkalinity']*1e-3)
                    OmegaA_sc.append(output['saturation_aragonite'])
                    OmegaC_sc.append(output['saturation_calcite'])

                alk_sc_change = np.asarray(alk_sc) - alk_sc[0]
                dic_sc_change = np.asarray(dic_sc) - dic_sc[0]
                axes[i].plot(times[i][j],(dic_change)*1000.,color='black',linewidth=2,linestyle='-.')
                axes[i].plot(times[i][j],(-1./0.59*co3_change)*1000.,linewidth=2,color='midnightblue',alpha=alphas[j],label='-'+str(3+3*j)+' W/m$^2$')
                axes[i].fill_between(times[i][j],(-1./0.49*co3_change)*1000.,(-1./0.69*co3_change)*1000.,color='midnightblue',alpha=0.5*alphas[j])
                axes[i].plot(times[i][j],(dic_sc_change)*1000.,linewidth=2,color='peru',label='-'+str(3+3*j)+' W/m$^2$')
                axes[i].plot(times[i][j],(-1./0.59*co3_change+alk_sc_change)*1000.,linewidth=4,color='forestgreen',alpha=0.7,label='$f'+labels[labeli[i]].upper()+'$')
                axes[i].fill_between(times[i][j],(-1./0.49*co3_change+alk_sc_change)*1000.,(-1./0.69*co3_change+alk_sc_change)*1000., color='forestgreen', alpha=0.3)
        else:
            dic_change = dic[i]-dic[i][0]
            alk_change = alk[i]-alk[i][0]
            co3_change = co3[i]-co3[i][0]
            output1 = pyco2.sys(par1=alk[i]*1e3, par1_type=1, opt_pH_scale=1, par2=dic[i]*1e3, par2_type=2,temperature=t[i],salinity=s[i],pressure=pr[i]/10000.,total_phosphate=po4[i]*1e3,total_silicate=sio[i]*1e3,opt_k_carbonic=14,opt_total_borate=2)
            ph[i] = output1['pH']
            co3[i] = output1['CO3']*1e-3
            
            if opt == 'amoc':
                axb = ax.twinx()
                ax2b = ax2.twinx()
                ax3b = ax3.twinx()
                ax.plot(times[i],amoc[i],alpha=alphas[i],linewidth=4,color='firebrick')
                axb.plot(times[i],temp[i],alpha=alphas[i],linewidth=4,color='midnightblue')
            
            if opt == 'co3ph':
                unc = pyco2.uncertainty_OEDG18
                unc["par1"] = 0.05 #0.0 #0.05
                unc["par2"] = 5.0 #0.0 #5.0
                unc["temperature"] = 0.1 #0.0 #0.1?
                output = pyco2.sys(par1=ph[i], par1_type=3, opt_pH_scale=1, par2=co3[i]*1e3, par2_type=6,temperature=t[i],salinity=[s[i][0] for k in range(len(sio[i]))],pressure=[pr[i][0]/10000. for k in range(len(sio[i]))],total_phosphate=[po4[i][0]*1e3 for k in range(len(sio[i]))],total_silicate=[sio[i][0]*1e3 for k in range(len(sio[i]))],opt_k_carbonic=14,opt_total_borate=2,uncertainty_into=["dic",],uncertainty_from=unc)
                dic_sc= output['dic']*1e-3
                udic_sc=output['u_dic']*1e-3
                alk_sc=output['alkalinity']*1e-3
                output = pyco2.sys(par1=ph[i], par1_type=3, opt_pH_scale=1, par2=co3[i]*1e3, par2_type=6,temperature=t[i],salinity=s[i],pressure=[pr[i][0]/10000. for k in range(len(sio[i]))],total_phosphate=[po4[i][0]*1e3 for k in range(len(sio[i]))],total_silicate=[sio[i][0]*1e3 for k in range(len(sio[i]))],opt_k_carbonic=14,opt_total_borate=2,uncertainty_into=["dic",],uncertainty_from=unc)
                dic_sc2=output['dic']*1e-3
                udic_sc2=output['u_dic']*1e-3
                alk_sc2=output['alkalinity']*1e-3

            if opt == 'co3':
                rec1 = [(-1./0.53*(co3_change+0.005))*1000.,(-1./0.53*(co3_change-0.005))*1000.,(-1./0.60*(co3_change-0.005))*1000.,(-1./0.60*(co3_change+0.005))*1000.]
                rec2 = [(-1./0.53*(co3_change+0.005)+alk_change)*1000.,(-1./0.53*(co3_change-0.005)+alk_change)*1000.,(-1./0.60*(co3_change-0.005)+alk_change)*1000.,(-1./0.60*(co3_change+0.005)+alk_change)*1000.]
                axes[i].plot(times[i],(-1./0.59*co3_change)*1000.,linewidth=4,color='midnightblue',alpha=0.7,label='$f'+labels[labeli[i]].upper()+'$')
                axes[i].fill_between(times[i],np.asarray(rec1).min(axis=0),np.asarray(rec1).max(axis=0), color='midnightblue', alpha=0.3)
                axes[i].plot(times[i],(-1./0.59*co3_change+alk_change)*1000.,linewidth=4,color='purple',alpha=0.7,label='$f'+labels[labeli[i]].upper()+'$')
                axes[i].fill_between(times[i],np.asarray(rec2).min(axis=0),np.asarray(rec2).max(axis=0), color='purple', alpha=0.3)
                axes[i].plot(times[i],(dic_change)*1000.,linestyle=':',color='black',linewidth=4)
            
            elif opt == 'co3ph':
                alk_sc_change = np.asarray(alk_sc) - alk_sc[0]
                dic_sc_change = np.asarray(dic_sc) - dic_sc[0]
                alk_sc_change2 = np.asarray(alk_sc2) - alk_sc2[0]
                dic_sc_change2 = np.asarray(dic_sc2) - dic_sc2[0]
                axes[i].plot(times[i],(dic_sc_change)*1000.,linewidth=4,color='forestgreen',alpha=0.7,label='$f'+labels[labeli[i]].upper()+'$')
                axes[i].fill_between(times[i],(dic_sc_change-udic_sc)*1000.,(dic_sc_change+udic_sc)*1000., color='forestgreen', alpha=0.3)
                axes[i].plot(times[i],(dic_sc_change2)*1000.,linewidth=4,color='peru',alpha=0.7,label='$f'+labels[labeli[i]].upper()+'$')
                axes[i].fill_between(times[i],(dic_sc_change2-udic_sc2)*1000.,(dic_sc_change2+udic_sc2)*1000., color='peru', alpha=0.3)
                axes[i].plot(times[i],(dic_change)*1000.,linestyle=':',color='black',linewidth=5)
                
            elif opt == 'aou':
                aou_change = np.asarray(aou[i])*117/170. - aou[i][0]*117/170.
                tou_change = np.asarray(tou[i])*117/170. - tou[i][0]*117/170.
                o2_change = -(np.asarray(o2[i]) - o2[i][0])*117/170.
                rem_change = -(-o2_change-(tou_change-aou_change))
                d13creg = -24.
                d13c_change = ((d13c[i][0]-d13creg)*dic[i][0]/(np.asarray(d13c[i])-d13creg)-dic[i][0])
                d13creg = -12.
                d13c_change2 = ((d13c[i][0]-d13creg)*dic[i][0]/(np.asarray(d13c[i])-d13creg)-dic[i][0])

                po4reg_change = np.asarray(po4reg) - po4reg[0]
                axes[i].plot(times[i],(aou_change)*1000,linestyle=':',color='gray',linewidth=5)
                axes[i].fill_between(times[i],(aou_change-0.017)*1000.,(aou_change+0.017)*1000., color='gray', alpha=0.3)
                axes[i].plot(times[i],(d13c_change)*1000,color='brown',linewidth=5)
                axes[i].fill_between(times[i],(d13c_change*1000-0.2),(d13c_change*1000+0.2), color='brown', alpha=0.3)
                axes[i].plot(times[i],(d13c_change)*1000,color='brown',linestyle=':',linewidth=5)
                axes[i].fill_between(times[i],(d13c_change*1000-0.2),(d13c_change*1000+0.2), color='brown', alpha=0.3)
                axes[i].plot(times[i],(dic_change)*1000,linestyle=':',color='black',linewidth=5)

            elif opt == 'aoureg':
                aou_change = np.asarray(aou[i])*117/170. - aou[i][0]*117/170.
                tou_change = np.asarray(tou[i])*117/170. - tou[i][0]*117/170.
                o2_change = -(np.asarray(o2[i]) - o2[i][0])*117/170.
                rem_change = -(-o2_change-(tou_change-aou_change))
                d13creg = -24.
                d13c_change = ((d13c[i][0]-d13creg)*dic[i][0]/(np.asarray(d13c[i])-d13creg)-dic[i][0])
                d13creg = -12.
                d13c_change2 = ((d13c[i][0]-d13creg)*dic[i][0]/(np.asarray(d13c[i])-d13creg)-dic[i][0])

                po4reg_change = np.asarray(po4reg[i]) - po4reg[i][0]
                axes[i].plot(times[i],(aou_change)*1000,linestyle=':',color='gray',linewidth=5)
                axes[i].fill_between(times[i],(aou_change-0.017)*1000.,(aou_change+0.017)*1000., color='gray', alpha=0.3)
                axes[i].plot(times[i],(tou_change)*1000,color='darkturquoise',linewidth=5)
                axes[i].fill_between(times[i],(tou_change-0.017)*1000.,(tou_change+0.017)*1000., color='darkturquoise', alpha=0.3)
                axes[i].plot(times[i],(d13c_change)*1000,color='brown',linewidth=5)
                axes[i].fill_between(times[i],(d13c_change*1000-0.2),(d13c_change*1000+0.2), color='brown', alpha=0.3)
                axes[i].plot(times[i],(d13c_change)*1000,color='brown',linewidth=5)
                axes[i].fill_between(times[i],(d13c_change*1000-0.2),(d13c_change*1000+0.2), color='brown', alpha=0.3)
                axes[i].plot(times[i],(po4reg_change)*117.*1000,linestyle=':',color='black',linewidth=5)
                
            elif opt == 's':
                axes[i].plot(times[i],(s[i]-s[i][0]),color='midnightblue',linewidth=5)
            elif opt == 'alk':
                if i < 3:
                    axes[i].plot(times[i],(alk[i]-alk[i][0])*1000,color='peru',linewidth=5)
                else:
                    axes[i-3].plot(times[i],(alk[i]-alk[i][0])*1000,color='midnightblue',linewidth=5)
    
    for i in range(len(axes)):
        axes[i].grid(ls=':',dashes=(1,10),color='black',lw=0.7,alpha=0.9)
        if '_C_' in region:
            axes[i].set_xlim([0,9.999])
            if '_EP' in region:
                axes[i].set_ylim([-50,200])
            else:
                axes[i].set_ylim([-100,150])
        else:
            axes[i].set_xlim([0,999])
            if opt == 'co3':
                axes[i].set_ylim([-175,175])
            elif opt == 'co3ph':
                axes[i].set_ylim([-400,400])
            elif opt == 'aou' or opt == 'aoureg':
                axes[i].set_ylim([-50,125])
                
    ax2.tick_params(axis='y',labelcolor='none')
    ax3.tick_params(axis='y',labelcolor='none')
    ax.tick_params(axis='both', which='major', labelsize=16)
    ax2.tick_params(axis='both', which='major', labelsize=16)
    ax3.tick_params(axis='both', which='major', labelsize=16)
    ax2.set_xlabel('Simulation time (yr)',fontsize=20)
    if opt == 'aoureg':
        ax.set_ylabel('DIC$_{reg}$ change (mmol/m$^3$)',fontsize=20)
    else:
        ax.set_ylabel('DIC change (mmol/m$^3$)',fontsize=20)
    if opt == 's':
        ax.set_ylabel('S change (PSU)',fontsize=20)
    elif opt == 'alk':
        ax.set_ylabel('ALK change (mmol/m$^3$)',fontsize=20)
    elif opt == 'amoc':
        ax.set_ylabel('AMOC (Sv)',fontsize=23)
        ax.yaxis.label.set_color('firebrick')
        ax.spines['left'].set_color('firebrick')
        ax.tick_params(axis='y', colors='firebrick')
        ax3b.set_ylabel('GMT ($^{\circ}$C)',fontsize=23)
        ax3b.yaxis.label.set_color('midnightblue')
        ax3b.spines['right'].set_color('midnightblue')
        ax3b.tick_params(axis='y', colors='midnightblue')
        axb.tick_params(axis='y',labelcolor='none')
        ax2b.tick_params(axis='y',labelcolor='none')

    marker0 = mlines.Line2D([0], [1],color='black',linestyle=':',linewidth=4,label='Simulation')
    markera = mlines.Line2D([0], [1],color='black',alpha=0.0,linestyle=':',linewidth=4,label='Simulation')
    marker1 = mlines.Line2D([0], [1],color='midnightblue',alpha=0.7,linewidth=4,label='Prediction A')
    marker2 = mlines.Line2D([0], [1],color='forestgreen',alpha=0.7,linewidth=4,label='Prediction B')
    marker3 = mlines.Line2D([0], [1],color='peru',alpha=0.7,linewidth=4,label='Prediction C')
    marker4 = mlines.Line2D([0], [1],color='purple',alpha=0.7,linewidth=4,label='Prediction D')
    marker5 = mlines.Line2D([0], [1],color='gray',alpha=0.7,linestyle=':',linewidth=4,label='Prediction D')
    marker6 = mlines.Line2D([0], [1],color='darkturquoise',alpha=0.7,linewidth=4,label='Prediction D')
    marker7 = mlines.Line2D([0], [1],color='brown',alpha=0.7,linewidth=4,label='Prediction D')
    if 'NA' in region:
        if opt == 'co3':
            ax.legend(handles=[marker0,markera,marker1,marker4],labels=['Simulation','Estimation from:','CO$_3^{2-}$','CO$_3^{2-}$+$\Delta$ALK'],loc=3,bbox_to_anchor=(-0.1,1.0),ncol=4,fontsize=20)
        elif opt == 'co3ph':
            ax.legend(handles=[marker0,markera,marker2,marker3],labels=['Simulation','Estimation from:','CO$_3^{2-}$,pH,T','CO$_3^{2-}$,pH,T + S'],loc=3,bbox_to_anchor=(-0.1,1.0),ncol=4,fontsize=20)
        elif opt == 'aou' or opt == 'aoureg':
            ax.legend(handles=[marker0,markera,marker5,marker7],labels=['Simulation','Estimation from:','AOU','$\delta^{13}$C'],loc=3,bbox_to_anchor=(-0.1,1.0),ncol=5,fontsize=20)
        elif opt == 'alk':
            ax.legend(handles=[marker3,marker1],labels=['with sediments','w/o sediments'],loc=3,bbox_to_anchor=(-0.1,1.0),ncol=2,fontsize=20)

    plt.subplots_adjust(left=0.11,bottom=0.15,wspace=0.1,top=0.85)
    plt.savefig(region+'_'+opt+'.pdf')

    plt.close()

In [18]:
# Produce Fig. 1-3
# Select whether to plot results from step change or glacial cycle simulations
#For step change results un-comment:
ens = 'step'
#For glacial cycle results un-comment:
ens = 'cycle'

# Select whether to plot results from simulations with or without interactive sediments
seds = 'with'
#seds = 'without'

# Select depth layer to plot
depth = 'ben' #'ben' or 'surf' or '100410' or '4102000'

if ens == 'step':
    if depth == 'ben':
        runsA = {'NA':NA_full,'NP':NP_full,'EP':EP_full,'SO':SO_full,'GL':GL_full}
        runsC = {'NA':NA_C_full,'NP':NP_C_full,'EP':EP_C_full,'SO':SO_C_full,'GL':GL_C_full}
    elif depth == 'surf':
        runsA = {'NA':NA_surf_full,'NP':NP_surf_full,'EP':EP_surf_full,'SO':SO_surf_full,'GL':GL_surf_full}
        runsC = {'NA':NA_surf_C_full,'NP':NP_surf_C_full,'EP':EP_surf_C_full,'SO':SO_surf_C_full,'GL':GL_surf_C_full}
    elif depth == '100410':
        runsA = {'NA':NA_100410_full,'NP':NP_100410_full,'EP':EP_100410_full,'SO':SO_100410_full,'GL':GL_100410_full}
        runsC = {'NA':NA_100410_C_full,'NP':NP_100410_C_full,'EP':EP_100410_C_full,'SO':SO_100410_C_full,'GL':GL_100410_C_full}
    elif depth == '4102000':
        runsA = {'NA':NA_4102000_full,'NP':NP_4102000_full,'EP':EP_4102000_full,'SO':SO_4102000_full,'GL':GL_4102000_full}
        runsC = {'NA':NA_4102000_C_full,'NP':NP_4102000_C_full,'EP':EP_4102000_C_full,'SO':SO_4102000_C_full,'GL':GL_4102000_C_full}
elif ens == 'cycle':
    runsA = {'NA':NA_longruns_full,'NP':NP_longruns_full,'EP':EP_longruns_full,'SO':SO_longruns_full,'GL':GL_longruns_full}
    runsC = {'NA':NA_longruns_C_full,'NP':NP_longruns_C_full,'EP':EP_longruns_C_full,'SO':SO_longruns_C_full,'GL':GL_longruns_C_full}

for region in ['NA','NP','SO','EP','GL']:
    if seds == 'with':
        profiles = runsA[region]
        times = runsA['time']
    elif seds == 'without':
        profiles = runsC[region]
        times = runsC['time']
    
    varsn = ['ALK','CO3','DIC','PH','S','TEMP','PO4','SiO','Pr','AOU','TOU','PO4reg','O2','d13c']
    if ens == 'step':
        simn = ['remin','fw','caco','sowi','rad','phos']
    elif ens == 'cycle':
        simn = ['phys','allC',]

    #Figure 1
    timeseries_short_alkdicerrors_stack_sc(times,profiles['ALK'],profiles['DIC'],profiles['CO3'],profiles['PH'],profiles['S'],profiles['TEMP'],profiles['Pr'],profiles['PO4'],profiles['SiO'],profiles['PO4reg'],profiles['AOU'],profiles['TOU'],profiles['O2'],profiles['d13c'],'Figure1_'+region+'_'+depth,labels,'co3')
    #Figure 2
    timeseries_short_alkdicerrors_stack_sc(times,profiles['ALK'],profiles['DIC'],profiles['CO3'],profiles['PH'],profiles['S'],profiles['TEMP'],profiles['Pr'],profiles['PO4'],profiles['SiO'],profiles['PO4reg'],profiles['AOU'],profiles['TOU'],profiles['O2'],profiles['d13c'],'Figure2_'+region+'_'+depth,labels,'co3ph')
    #Figure 3
    timeseries_short_alkdicerrors_stack_sc(times,profiles['ALK'],profiles['DIC'],profiles['CO3'],profiles['PH'],profiles['S'],profiles['TEMP'],profiles['Pr'],profiles['PO4'],profiles['SiO'],profiles['PO4reg'],profiles['AOU'],profiles['TOU'],profiles['O2'],profiles['d13c'],'Figure3_'+region+'_'+depth,labels,'aou')
    #Figure 3 with regenerated DIC
    timeseries_short_alkdicerrors_stack_sc(times,profiles['ALK'],profiles['DIC'],profiles['CO3'],profiles['PH'],profiles['S'],profiles['TEMP'],profiles['Pr'],profiles['PO4'],profiles['SiO'],profiles['PO4reg'],profiles['AOU'],profiles['TOU'],profiles['O2'],profiles['d13c'],'Figure3reg_steps_'+region+'_'+depth,labels,'aoureg')
    #Salinity
    timeseries_short_alkdicerrors_stack_sc(times,profiles['ALK'],profiles['DIC'],profiles['CO3'],profiles['PH'],profiles['S'],profiles['TEMP'],profiles['Pr'],profiles['PO4'],profiles['SiO'],profiles['PO4reg'],profiles['AOU'],profiles['TOU'],profiles['O2'],profiles['d13c'],'Salinity_'+region+'_'+depth,labels,'s')


NA done
NP done
SO done
EP done
GL done


In [ ]:
#### read in output of simulations of multiple glacial cycles from Adloff et al. (2024), Clim. Past###
#### The simulation output can be downloaded from https://doi.org/10.5281/zenodo.11385608 and need to be unpacked###
path = './' # adjust path to the unpacked output files
ctrl_ra = xr.open_dataset(path+'base_2d_summary.nc', decode_times=False)
kgas_ra = xr.open_dataset(path+'kgas_2d_summary.nc', decode_times=False)
sowi_ra = xr.open_dataset(path+'sowi_2d_summary.nc', decode_times=False)
dust_ra = xr.open_dataset(path+'aero_2d_summary.nc', decode_times=False)
remin_ra = xr.open_dataset(path+'remi_2d_summary.nc', decode_times=False)
phos_ra = xr.open_dataset(path+'po4_2d_summary.nc', decode_times=False)
caco_ra = xr.open_dataset(path+'pipo_2d_summary.nc', decode_times=False)
land_ra = xr.open_dataset(path+'land_2d_summary.nc', decode_times=False)
co2t_ra = xr.open_dataset(path+'co2t_2d_summary.nc', decode_times=False)
phys_ra = xr.open_dataset(path+'phys_2d_summary.nc', decode_times=False)
bgc_ra = xr.open_dataset(path+'bgc_2d_summary.nc', decode_times=False)
allC_ra = xr.open_dataset(path+'all_2d_summary.nc', decode_times=False)

ctrl_raC = xr.open_dataset(path+'baseC_2d_summary.nc', decode_times=False)
kgas_raC = xr.open_dataset(path+'kgasC_2d_summary.nc', decode_times=False)
sowi_raC = xr.open_dataset(path+'sowiC_2d_summary.nc', decode_times=False)
dust_raC = xr.open_dataset(path+'aeroC_2d_summary.nc', decode_times=False)
remin_raC = xr.open_dataset(path+'remiC_2d_summary.nc', decode_times=False)
phos_raC = xr.open_dataset(path+'po4C_2d_summary.nc', decode_times=False)
caco_raC = xr.open_dataset(path+'pipoC_2d_summary.nc', decode_times=False)
land_raC = xr.open_dataset(path+'landC_2d_summary.nc', decode_times=False)
co2t_raC = xr.open_dataset(path+'co2tC_2d_summary.nc', decode_times=False)
phys_raC = xr.open_dataset(path+'physC_2d_summary.nc', decode_times=False)
bgc_raC = xr.open_dataset(path+'bgcC_2d_summary.nc', decode_times=False)
allC_raC = xr.open_dataset(path+'allC_2d_summary.nc', decode_times=False)

In [ ]:
#produce Figure 4
labels = {'std':'base','kgas':'kgas','sowi':'sowi','dust':'aero','phos':'po4','caco':'pipo','land':'land','remin':'remi','co2t':'ALK imbalance','bgc':'bgc','phys':'phys','allO':'allO','allC':'all'}
datasets_2d = {'std':ctrl_ra,'kgas':kgas_ra,'sowi':sowi_ra,'dust':dust_ra,'remin':remin_ra,'phos':phos_ra,'caco':caco_ra,'land':land_ra,'co2t':co2t_ra,'phys':phys_ra,'bgc':bgc_ra,'allO':allO_ra,'allC':allC_ra}
datasets_2dC = {'std':ctrl_raC,'kgas':kgas_raC,'sowi':sowi_raC,'dust':dust_raC,'remin':remin_raC,'phos':phos_raC,'caco':caco_raC,'land':land_raC,'co2t':co2t_raC,'phys':phys_raC,'bgc':bgc_raC,'allO':allO_ra,'allC':allC_raC}

def scatter_co2dic():
    colors = {'std':'olive','kgas':'blue','sowi':'lightblue','dust':'gray','phos':'xkcd:burnt yellow','caco':'xkcd:earth','land':'xkcd:tea','remin':'xkcd:greenish beige','co2t':'cyan','bgc':'xkcd:iris','phys':'xkcd:carmine','allO':'xkcd:gray pink','allC':'xkcd:gray pink'}
    var1s = []
    var2s = []
    var1Cs = []
    var2Cs = []
    markers = []
    sed = 'no' # set to 'yes' to produce figure for simulations that include interactive sediments, otherwise 'no'
    nosed = 'yes' # set to 'yes' to produce figure for simulations that include interactive sediments, otherwise 'no'
    if sed == 'yes':
        special_runs = ['remin','phos','land','co2t']
    elif nosed == 'yes':
        special_runs = ['land',]

    fig, ax = plt.subplots(figsize=(11,7))
    for j,run in enumerate(['std','kgas','sowi','dust','remin','phos','caco','land','co2t']):
        if run == 'std':
            #DIC
            var1 = (datasets_2d[run]['Cstock_DIC'].sel(time=slice(-125000,0)))/12
            var1C = (datasets_2dC[run]['Cstock_DIC'].sel(time=slice(-125000,0)))/12
            
            var2 = datasets_2d[run]['atmpco2'].sel(time=slice(-125000,0))
            var2C = datasets_2dC[run]['atmpco2'].sel(time=slice(-125000,0))
            
            var1std = var1
            var1Cstd = var1C
            var2std = var2
            var2Cstd = var2C
        else:
            var1 = (datasets_2d[run]['Cstock_DIC'].sel(time=slice(-125000,0)))/12-var1std
            var1C = (datasets_2dC[run]['Cstock_DIC'].sel(time=slice(-125000,0)))/12-var1Cstd
            
            var2 = datasets_2d[run]['atmpco2'].sel(time=slice(-125000,0))-var2std
            var2C = datasets_2dC[run]['atmpco2'].sel(time=slice(-125000,0))-var2Cstd

        if run in special_runs:
            if sed == 'yes':
                ax.scatter(var1,var2,marker='s',alpha=0.6,zorder=2,color=colors[run])
                if factorial == 'yes':
                    labelname = '$f'+labels[run].upper()+'$'
                else:
                    labelname = '$'+labels[run].upper()+'$'
                markers.append(mlines.Line2D([0], [0], marker='s', linewidth=0,color=colors[run],label=labelname))
            if not run == 'phos' and nosed=='yes':
                ax.scatter(var1C,var2C,marker='x',zorder=2,color=colors[run])
                if sed=='no':
                    if factorial == 'yes':
                        labelname = '$f'+labels[run].upper()+'$'
                    else:
                        labelname = '$'+labels[run].upper()+'$'
                    markers.append(mlines.Line2D([0], [0], marker='x', linewidth=0,color=colors[run],label=labelname))

        else:
            var1s.extend(var1-var1[-1])
            var2s.extend(var2-var2[-1])

                
    if len(var1s)>0 and sed=='yes':
        ax.scatter(var1s,var2s,marker='s',color='gray',alpha=0.6)
        markers.append(mlines.Line2D([0], [0], marker='o', linewidth=0,color='gray',label='other forcings'))
    if len(var1Cs)>0 and nosed == 'yes':
        ax.scatter(var1Cs,var2Cs,marker='x',color='gray',label='no sediments')
        if sed=='yes':
            markers.append(mlines.Line2D([0], [0], marker='o', linewidth=0,color='gray',label='other forcings'))
        elif sed=='no':
            markers.append(mlines.Line2D([0], [0], marker='o', linewidth=0,color='gray',label='other processes'))
            
    ax.plot([-170,170],[170,-170],linewidth=1,color='black')
    ax.annotate(text='1:-1',xy=(-167,170),fontsize=14,color='black')
    if sed=='yes' and nosed=='yes':
        markers.append(mlines.Line2D([0], [0], marker='s', linewidth=0,color='black',label='ocean+sediments'))
        markers.append(mlines.Line2D([0], [0], marker='x', linewidth=0,color='black',label='ocean only'))

    ax.legend(handles=markers,loc='best',fontsize=15)
    ax.grid(ls=':',dashes=(1,10),color='black',lw=0.7,alpha=0.9)

    ax.tick_params(axis='both', which='major', labelsize=15)
    ax.set_ylabel('$\Delta$atmospheric CO$_2$ (ppm)',fontsize=18)
    ax3 = ax.twinx()
    mn, mx = ax.get_ylim()
    ax3.set_ylim(mn*2.12/12., mx*2.12/12.)
    ax3.set_ylabel('$\Delta$atmospheric CO$_2$ (Pmol C)',fontsize=18)
    ax3.tick_params(axis='y', which='major', labelsize=15)
    ax.set_xlabel('$\Delta$DIC (Pmol C)',fontsize=18)
    ax4 = ax.twiny()
    mn, mx = ax.get_ylim()
    ax4.set_ylim(mn*1e15/1.35449639e18*1000, mx*1e15/1.35449639e18*1000)
    ax4.set_ylabel('$\Delta$DIC (mmol/m$^3$)',fontsize=18)
    ax4.tick_params(axis='y', which='major', labelsize=15)

    plt.savefig('Figure4.png')

scatter_co2dic()